In [46]:
import tensorflow as tf
import torch as torch
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np
from numpy import array
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt

In [8]:
data = pd.read_csv("~/PycharmProjects/Diabetes/final.csv", sep=',')

Albumin_Creatinine_data = data[['Patient_ID', 'PerformedDate', 'Albumin/Creatinine_Res']]
Albumin_Creatinine_data = Albumin_Creatinine_data.dropna()
Albumin_Creatinine_data = Albumin_Creatinine_data[Albumin_Creatinine_data['Albumin/Creatinine_Res'] < 300]
Albumin_Creatinine_data = Albumin_Creatinine_data.sort_values(by='PerformedDate', ascending=True)

grouped_patients = Albumin_Creatinine_data.groupby("Patient_ID")
grouped_lists = grouped_patients["Albumin/Creatinine_Res"].apply(list)
grouped_lists = grouped_lists.reset_index()
grouped_lists['Albumin/Creatinine_Res'] = grouped_lists['Albumin/Creatinine_Res'].apply(lambda x: list(set(x)))

data = grouped_lists[['Albumin/Creatinine_Res']]

def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence) - 1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

for index, row in data.iterrows() :
    raw_seq = data['Albumin/Creatinine_Res'].at[index]
    if len(data['Albumin/Creatinine_Res'].at[index]) > 8 :
        # define input sequence
# choose a number of time steps
        n_steps = 3
# split into samples
        X, y = split_sequence(raw_seq, n_steps)
# reshape from [samples, timesteps] into [samples, timesteps, features]
        n_features = 1
        X = X.reshape((X.shape[0], X.shape[1], n_features))
        print(X)
# define model
        model = Sequential()
        model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')
# fit model
        model.fit(X, y, epochs=200, verbose=0)
# demonstrate prediction
        x_input = np.asarray(raw_seq[-4:-1])
        x_input = x_input.reshape((1, n_steps, n_features))
        yhat = model.predict(x_input, verbose=0)
        y_actual = np.asarray(raw_seq[-1:])
        rms = sqrt(mean_squared_error(y_actual, yhat))
        print(rms)

    else :
        continue


[[[ 36.]
  [ 37.]
  [  5.]]

 [[ 37.]
  [  5.]
  [ 68.]]

 [[  5.]
  [ 68.]
  [  8.]]

 [[ 68.]
  [  8.]
  [ 12.]]

 [[  8.]
  [ 12.]
  [ 13.]]

 [[ 12.]
  [ 13.]
  [ 16.]]

 [[ 13.]
  [ 16.]
  [ 53.]]

 [[ 16.]
  [ 53.]
  [ 55.]]

 [[ 53.]
  [ 55.]
  [186.]]

 [[ 55.]
  [186.]
  [ 27.]]]
0.24385452270507812
[[[ 1.]
  [ 2.]
  [ 4.]]

 [[ 2.]
  [ 4.]
  [ 5.]]

 [[ 4.]
  [ 5.]
  [ 6.]]

 [[ 5.]
  [ 6.]
  [ 7.]]

 [[ 6.]
  [ 7.]
  [ 8.]]

 [[ 7.]
  [ 8.]
  [ 9.]]

 [[ 8.]
  [ 9.]
  [11.]]

 [[ 9.]
  [11.]
  [12.]]

 [[11.]
  [12.]
  [13.]]]
0.035162925720214844
[[[1.]
  [2.]
  [3.]]

 [[2.]
  [3.]
  [4.]]

 [[3.]
  [4.]
  [5.]]

 [[4.]
  [5.]
  [6.]]

 [[5.]
  [6.]
  [7.]]

 [[6.]
  [7.]
  [8.]]]
1.2300338745117188
[[[ 1.]
  [ 8.]
  [72.]]

 [[ 8.]
  [72.]
  [13.]]

 [[72.]
  [13.]
  [48.]]

 [[13.]
  [48.]
  [17.]]

 [[48.]
  [17.]
  [19.]]

 [[17.]
  [19.]
  [51.]]

 [[19.]
  [51.]
  [23.]]]
10.187847137451172
[[[32.]
  [34.]
  [ 5.]]

 [[34.]
  [ 5.]
  [41.]]

 [[ 5.]
  [41.]
  [45.]]


1.4135055541992188
[[[ 3.]
  [ 6.]
  [ 7.]]

 [[ 6.]
  [ 7.]
  [ 8.]]

 [[ 7.]
  [ 8.]
  [42.]]

 [[ 8.]
  [42.]
  [12.]]

 [[42.]
  [12.]
  [16.]]

 [[12.]
  [16.]
  [48.]]]
0.03761100769042969
[[[  1.]
  [  3.]
  [  5.]]

 [[  3.]
  [  5.]
  [  6.]]

 [[  5.]
  [  6.]
  [136.]]

 [[  6.]
  [136.]
  [ 11.]]

 [[136.]
  [ 11.]
  [ 15.]]

 [[ 11.]
  [ 15.]
  [ 19.]]

 [[ 15.]
  [ 19.]
  [ 22.]]

 [[ 19.]
  [ 22.]
  [ 24.]]

 [[ 22.]
  [ 24.]
  [ 29.]]

 [[ 24.]
  [ 29.]
  [ 33.]]

 [[ 29.]
  [ 33.]
  [161.]]

 [[ 33.]
  [161.]
  [ 35.]]

 [[161.]
  [ 35.]
  [ 54.]]

 [[ 35.]
  [ 54.]
  [ 76.]]

 [[ 54.]
  [ 76.]
  [216.]]

 [[ 76.]
  [216.]
  [ 89.]]

 [[216.]
  [ 89.]
  [239.]]

 [[ 89.]
  [239.]
  [241.]]]
16.16619873046875
[[[ 33.]
  [  6.]
  [  7.]]

 [[  6.]
  [  7.]
  [  8.]]

 [[  7.]
  [  8.]
  [266.]]

 [[  8.]
  [266.]
  [ 44.]]

 [[266.]
  [ 44.]
  [ 13.]]

 [[ 44.]
  [ 13.]
  [205.]]

 [[ 13.]
  [205.]
  [ 16.]]

 [[205.]
  [ 16.]
  [ 17.]]

 [[ 16.]
  [ 17.]
  [ 18.]]

 [[ 

0.17664337158203125
[[[256.]
  [ 34.]
  [  2.]]

 [[ 34.]
  [  2.]
  [ 36.]]

 [[  2.]
  [ 36.]
  [101.]]

 [[ 36.]
  [101.]
  [  7.]]

 [[101.]
  [  7.]
  [ 46.]]

 [[  7.]
  [ 46.]
  [ 15.]]

 [[ 46.]
  [ 15.]
  [ 14.]]

 [[ 15.]
  [ 14.]
  [ 19.]]]
0.016971588134765625
[[[128.]
  [  2.]
  [  3.]]

 [[  2.]
  [  3.]
  [ 34.]]

 [[  3.]
  [ 34.]
  [  8.]]

 [[ 34.]
  [  8.]
  [ 41.]]

 [[  8.]
  [ 41.]
  [107.]]

 [[ 41.]
  [107.]
  [ 76.]]

 [[107.]
  [ 76.]
  [ 77.]]

 [[ 76.]
  [ 77.]
  [ 79.]]

 [[ 77.]
  [ 79.]
  [ 17.]]

 [[ 79.]
  [ 17.]
  [ 83.]]]
0.10282707214355469
[[[ 35.]
  [  3.]
  [  5.]]

 [[  3.]
  [  5.]
  [131.]]

 [[  5.]
  [131.]
  [  7.]]

 [[131.]
  [  7.]
  [  8.]]

 [[  7.]
  [  8.]
  [ 73.]]

 [[  8.]
  [ 73.]
  [ 16.]]

 [[ 73.]
  [ 16.]
  [ 17.]]

 [[ 16.]
  [ 17.]
  [ 18.]]

 [[ 17.]
  [ 18.]
  [ 87.]]]
0.002685546875
[[[  2.]
  [  5.]
  [  6.]]

 [[  5.]
  [  6.]
  [  7.]]

 [[  6.]
  [  7.]
  [  8.]]

 [[  7.]
  [  8.]
  [  9.]]

 [[  8.]
  [  9.]
  [ 11.

0.44831085205078125
[[[288.]
  [162.]
  [259.]]

 [[162.]
  [259.]
  [292.]]

 [[259.]
  [292.]
  [258.]]

 [[292.]
  [258.]
  [268.]]

 [[258.]
  [268.]
  [269.]]

 [[268.]
  [269.]
  [238.]]

 [[269.]
  [238.]
  [241.]]

 [[238.]
  [241.]
  [ 18.]]

 [[241.]
  [ 18.]
  [184.]]]
0.216796875
[[[ 98.]
  [194.]
  [  8.]]

 [[194.]
  [  8.]
  [168.]]

 [[  8.]
  [168.]
  [ 10.]]

 [[168.]
  [ 10.]
  [141.]]

 [[ 10.]
  [141.]
  [ 46.]]

 [[141.]
  [ 46.]
  [ 47.]]

 [[ 46.]
  [ 47.]
  [238.]]

 [[ 47.]
  [238.]
  [ 13.]]

 [[238.]
  [ 13.]
  [115.]]

 [[ 13.]
  [115.]
  [ 54.]]

 [[115.]
  [ 54.]
  [119.]]

 [[ 54.]
  [119.]
  [153.]]

 [[119.]
  [153.]
  [124.]]]
0.6565093994140625
[[[128.]
  [  3.]
  [  4.]]

 [[  3.]
  [  4.]
  [  5.]]

 [[  4.]
  [  5.]
  [  6.]]

 [[  5.]
  [  6.]
  [  7.]]

 [[  6.]
  [  7.]
  [  8.]]

 [[  7.]
  [  8.]
  [ 13.]]

 [[  8.]
  [ 13.]
  [ 14.]]

 [[ 13.]
  [ 14.]
  [ 15.]]

 [[ 14.]
  [ 15.]
  [ 19.]]

 [[ 15.]
  [ 19.]
  [ 51.]]

 [[ 19.]
  [ 51.]
  [

3.0859546661376953
[[[129.]
  [  2.]
  [ 97.]]

 [[  2.]
  [ 97.]
  [  5.]]

 [[ 97.]
  [  5.]
  [  7.]]

 [[  5.]
  [  7.]
  [  9.]]

 [[  7.]
  [  9.]
  [ 11.]]

 [[  9.]
  [ 11.]
  [ 12.]]

 [[ 11.]
  [ 12.]
  [145.]]

 [[ 12.]
  [145.]
  [ 18.]]

 [[145.]
  [ 18.]
  [ 24.]]]
1.6785888671875
[[[ 1.]
  [33.]
  [ 2.]]

 [[33.]
  [ 2.]
  [ 4.]]

 [[ 2.]
  [ 4.]
  [ 7.]]

 [[ 4.]
  [ 7.]
  [ 8.]]

 [[ 7.]
  [ 8.]
  [ 9.]]

 [[ 8.]
  [ 9.]
  [12.]]]
2.0022029876708984
[[[ 2.]
  [ 5.]
  [ 6.]]

 [[ 5.]
  [ 6.]
  [72.]]

 [[ 6.]
  [72.]
  [41.]]

 [[72.]
  [41.]
  [14.]]

 [[41.]
  [14.]
  [21.]]

 [[14.]
  [21.]
  [22.]]

 [[21.]
  [22.]
  [23.]]]
2.4265995025634766
[[[ 96.]
  [  1.]
  [ 37.]]

 [[  1.]
  [ 37.]
  [198.]]

 [[ 37.]
  [198.]
  [  8.]]

 [[198.]
  [  8.]
  [  9.]]

 [[  8.]
  [  9.]
  [109.]]

 [[  9.]
  [109.]
  [ 15.]]

 [[109.]
  [ 15.]
  [ 16.]]

 [[ 15.]
  [ 16.]
  [245.]]]
0.9049625396728516
[[[ 2.]
  [ 5.]
  [ 8.]]

 [[ 5.]
  [ 8.]
  [ 9.]]

 [[ 8.]
  [ 9.]
  [12.]]


...

0.56085205078125
[[[  1.]
  [  3.]
  [  6.]]

 [[  3.]
  [  6.]
  [  7.]]

 [[  6.]
  [  7.]
  [  8.]]

 [[  7.]
  [  8.]
  [ 12.]]

 [[  8.]
  [ 12.]
  [ 14.]]

 [[ 12.]
  [ 14.]
  [ 15.]]

 [[ 14.]
  [ 15.]
  [207.]]

 [[ 15.]
  [207.]
  [ 17.]]

 [[207.]
  [ 17.]
  [ 19.]]

 [[ 17.]
  [ 19.]
  [ 21.]]

 [[ 19.]
  [ 21.]
  [ 55.]]

 [[ 21.]
  [ 55.]
  [221.]]]
2.586517333984375
[[[  3.]
  [  4.]
  [196.]]

 [[  4.]
  [196.]
  [ 43.]]

 [[196.]
  [ 43.]
  [ 78.]]

 [[ 43.]
  [ 78.]
  [ 79.]]

 [[ 78.]
  [ 79.]
  [ 48.]]

 [[ 79.]
  [ 48.]
  [ 52.]]

 [[ 48.]
  [ 52.]
  [ 55.]]]
0.01226043701171875
[[[ 32.]
  [  1.]
  [  5.]]

 [[  1.]
  [  5.]
  [  6.]]

 [[  5.]
  [  6.]
  [ 12.]]

 [[  6.]
  [ 12.]
  [ 46.]]

 [[ 12.]
  [ 46.]
  [ 14.]]

 [[ 46.]
  [ 14.]
  [207.]]

 [[ 14.]
  [207.]
  [ 18.]]

 [[207.]
  [ 18.]
  [ 22.]]

 [[ 18.]
  [ 22.]
  [ 87.]]

 [[ 22.]
  [ 87.]
  [ 24.]]

 [[ 87.]
  [ 24.]
  [ 59.]]

 [[ 24.]
  [ 59.]
  [ 29.]]]
20.307476043701172
[[[  1.]
  [ 35.]
  [  4.]]

2.1698150634765625
[[[265.]
  [ 11.]
  [ 12.]]

 [[ 11.]
  [ 12.]
  [ 51.]]

 [[ 12.]
  [ 51.]
  [ 23.]]

 [[ 51.]
  [ 23.]
  [ 57.]]

 [[ 23.]
  [ 57.]
  [124.]]

 [[ 57.]
  [124.]
  [ 93.]]

 [[124.]
  [ 93.]
  [254.]]]
0.017848968505859375
[[[224.]
  [ 65.]
  [  2.]]

 [[ 65.]
  [  2.]
  [  3.]]

 [[  2.]
  [  3.]
  [  4.]]

 [[  3.]
  [  4.]
  [  5.]]

 [[  4.]
  [  5.]
  [  6.]]

 [[  5.]
  [  6.]
  [ 39.]]

 [[  6.]
  [ 39.]
  [  8.]]

 [[ 39.]
  [  8.]
  [  9.]]

 [[  8.]
  [  9.]
  [ 42.]]

 [[  9.]
  [ 42.]
  [ 77.]]

 [[ 42.]
  [ 77.]
  [ 83.]]]
0.13124847412109375
[[[  1.]
  [ 33.]
  [  3.]]

 [[ 33.]
  [  3.]
  [  5.]]

 [[  3.]
  [  5.]
  [  6.]]

 [[  5.]
  [  6.]
  [  7.]]

 [[  6.]
  [  7.]
  [299.]]

 [[  7.]
  [299.]
  [203.]]

 [[299.]
  [203.]
  [ 13.]]

 [[203.]
  [ 13.]
  [205.]]

 [[ 13.]
  [205.]
  [245.]]

 [[205.]
  [245.]
  [151.]]

 [[245.]
  [151.]
  [ 58.]]]
3.837657928466797
[[[161.]
  [ 97.]
  [ 65.]]

 [[ 97.]
  [ 65.]
  [ 98.]]

 [[ 65.]
  [ 98.]
  [  

0.15375137329101562
[[[  1.]
  [  5.]
  [  7.]]

 [[  5.]
  [  7.]
  [  9.]]

 [[  7.]
  [  9.]
  [ 12.]]

 [[  9.]
  [ 12.]
  [ 13.]]

 [[ 12.]
  [ 13.]
  [ 14.]]

 [[ 13.]
  [ 14.]
  [ 15.]]

 [[ 14.]
  [ 15.]
  [ 16.]]

 [[ 15.]
  [ 16.]
  [ 17.]]

 [[ 16.]
  [ 17.]
  [ 18.]]

 [[ 17.]
  [ 18.]
  [ 19.]]

 [[ 18.]
  [ 19.]
  [ 21.]]

 [[ 19.]
  [ 21.]
  [152.]]

 [[ 21.]
  [152.]
  [ 24.]]

 [[152.]
  [ 24.]
  [ 37.]]

 [[ 24.]
  [ 37.]
  [ 61.]]

 [[ 37.]
  [ 61.]
  [ 65.]]

 [[ 61.]
  [ 65.]
  [215.]]

 [[ 65.]
  [215.]
  [ 94.]]

 [[215.]
  [ 94.]
  [ 97.]]

 [[ 94.]
  [ 97.]
  [107.]]

 [[ 97.]
  [107.]
  [113.]]]
4.1212005615234375
[[[  1.]
  [  8.]
  [  9.]]

 [[  8.]
  [  9.]
  [ 11.]]

 [[  9.]
  [ 11.]
  [ 12.]]

 [[ 11.]
  [ 12.]
  [ 13.]]

 [[ 12.]
  [ 13.]
  [ 15.]]

 [[ 13.]
  [ 15.]
  [ 16.]]

 [[ 15.]
  [ 16.]
  [ 17.]]

 [[ 16.]
  [ 17.]
  [ 18.]]

 [[ 17.]
  [ 18.]
  [178.]]

 [[ 18.]
  [178.]
  [ 21.]]]
0.9169559478759766
[[[129.]
  [101.]
  [ 75.]]

 [[101.]
  [ 7

0.3911399841308594
[[[  1.]
  [162.]
  [  4.]]

 [[162.]
  [  4.]
  [  6.]]

 [[  4.]
  [  6.]
  [  8.]]

 [[  6.]
  [  8.]
  [  9.]]

 [[  8.]
  [  9.]
  [ 42.]]

 [[  9.]
  [ 42.]
  [106.]]

 [[ 42.]
  [106.]
  [105.]]

 [[106.]
  [105.]
  [ 13.]]

 [[105.]
  [ 13.]
  [ 18.]]

 [[ 13.]
  [ 18.]
  [182.]]

 [[ 18.]
  [182.]
  [184.]]

 [[182.]
  [184.]
  [153.]]]
0.0718231201171875
[[[  3.]
  [  7.]
  [231.]]

 [[  7.]
  [231.]
  [ 13.]]

 [[231.]
  [ 13.]
  [ 14.]]

 [[ 13.]
  [ 14.]
  [ 16.]]

 [[ 14.]
  [ 16.]
  [208.]]

 [[ 16.]
  [208.]
  [214.]]]
1.4919204711914062
[[[ 2.]
  [ 3.]
  [36.]]

 [[ 3.]
  [36.]
  [ 6.]]

 [[36.]
  [ 6.]
  [16.]]

 [[ 6.]
  [16.]
  [23.]]

 [[16.]
  [23.]
  [24.]]

 [[23.]
  [24.]
  [57.]]

 [[24.]
  [57.]
  [28.]]]
0.006439208984375
[[[ 2.]
  [ 5.]
  [ 6.]]

 [[ 5.]
  [ 6.]
  [ 7.]]

 [[ 6.]
  [ 7.]
  [ 8.]]

 [[ 7.]
  [ 8.]
  [12.]]

 [[ 8.]
  [12.]
  [13.]]

 [[12.]
  [13.]
  [14.]]

 [[13.]
  [14.]
  [18.]]

 [[14.]
  [18.]
  [19.]]

 [[18.]
  [19

KeyboardInterrupt: 

In [4]:
data = pd.read_csv("~/PycharmProjects/Diabetes/final.csv", sep=',')

Albumin_Creatinine_data = data[['Patient_ID', 'PerformedDate', 'Albumin/Creatinine_Res']]
Albumin_Creatinine_data = Albumin_Creatinine_data.dropna()
Albumin_Creatinine_data = Albumin_Creatinine_data.sort_values(by='PerformedDate', ascending=True)

grouped_patients = Albumin_Creatinine_data.groupby("Patient_ID")
grouped_lists = grouped_patients["Albumin/Creatinine_Res"].apply(list)
grouped_lists = grouped_lists.reset_index()
grouped_lists['Albumin/Creatinine_Res'] = grouped_lists['Albumin/Creatinine_Res'].apply(lambda x: list(set(x)))
data = grouped_lists[['Albumin/Creatinine_Res']]

def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence) - 1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

for index, row in data.iterrows() :
    raw_seq = data['Albumin/Creatinine_Res'].at[index]
    if len(data['Albumin/Creatinine_Res'].at[index]) > 8 :
        n_steps = 3
# split into samples
        X, y = split_sequence(raw_seq, n_steps)
# reshape from [samples, timesteps] into [samples, timesteps, features]
        n_features = 1
        X = X.reshape((X.shape[0], X.shape[1], n_features))
        x_input = np.asarray(raw_seq[-4:-1])
        x_input = x_input.reshape((1, n_steps, n_features))
        y_actual = np.asarray(raw_seq[-1:])
    else :
        continue

In [ ]:
data = pd.read_csv("~/PycharmProjects/Diabetes/final.csv", sep=',')

Albumin_Creatinine_data = data[['Patient_ID', 'PerformedDate', 'Albumin/Creatinine_Res']]
Albumin_Creatinine_data = Albumin_Creatinine_data.dropna()
Albumin_Creatinine_data = Albumin_Creatinine_data[Albumin_Creatinine_data['Albumin/Creatinine_Res'] < 300]
Albumin_Creatinine_data = Albumin_Creatinine_data.sort_values(by='PerformedDate', ascending=True)
print(Albumin_Creatinine_data)

In [ ]:
data = pd.read_csv("~/PycharmProjects/Diabetes/final.csv", sep=',')

Albumin_Creatinine_data = data[['Patient_ID', 'PerformedDate', 'Albumin/Creatinine_Res']]
Albumin_Creatinine_data = Albumin_Creatinine_data.dropna()
Albumin_Creatinine_data = Albumin_Creatinine_data[Albumin_Creatinine_data['Albumin/Creatinine_Res'] < 300]
Albumin_Creatinine_data = Albumin_Creatinine_data.sort_values(by='PerformedDate', ascending=True)

grouped_patients = Albumin_Creatinine_data.groupby("Patient_ID")
grouped_lists = grouped_patients["Albumin/Creatinine_Res"].apply(list)
grouped_lists = grouped_lists.reset_index()
grouped_lists['Albumin/Creatinine_Res'] = grouped_lists['Albumin/Creatinine_Res'].apply(lambda x: list(set(x)))

data = grouped_lists[['Albumin/Creatinine_Res']]

def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence) - 1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

a = [0,0,0]
b = [[0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]
for index, row in data.iterrows() :
    #raw_seq = data['Albumin/Creatinine_Res'].at[index]
    if len(data['Albumin/Creatinine_Res'].at[index]) > 8 :
        seq = data['Albumin/Creatinine_Res'].at[index]
        # choose a number of time steps
        n_steps = 3
        # split into samples
        X, y = split_sequence(seq, n_steps)
        y = y.reshape((-1, 1))
        a = np.vstack((a, X))
        b = np.vstack((b, y))
    else:
        continue
X = np.delete(a, (0), axis=0)
y = b[10:, :]
print(y)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=0)

for index, row in data.iterrows() :
# demonstrate prediction
        x_input = np.asarray(seq[-4:-1])
        x_input = x_input.reshape((1, n_steps, n_features))
        yhat = model.predict(x_input, verbose=0)
        print(yhat)
        y_actual = np.asarray(seq[-1:])
        rms = sqrt(mean_squared_error(y_actual, yhat))
        print(rms)